In [25]:
import pandas as pd
import pickle

In [26]:
with open('vectors.pkl', 'rb') as f:
    vectors = pickle.load(f)

whole_df = pd.read_csv('preprocessed_data.csv')
whole_df.set_index('id', inplace=True)

non_normalized_df = pd.read_csv('non_normalized_data.csv')
non_normalized_df.set_index('id', inplace=True)

In [ ]:
target_variables = ['trstprl', 'badge', 'lrscale']

In [28]:
def drop_missing_values(df):
    return df[~df.isin([77, 88, 99]).any(axis=1)]

whole_df = drop_missing_values(whole_df)
non_normalized_df = drop_missing_values(non_normalized_df)

In [29]:
df = whole_df[whole_df.columns.difference(target_variables)]

In [30]:
def process_row(row):
    country_code = row.name[:2]
    row_vector = 0

    for col, value in row.items():
        if value==77:
            row_vector += vectors[col][country_code][1]
        elif value==88:
            row_vector += vectors[col][country_code][2]
        elif value==99:
            row_vector += vectors[col][country_code][3]
        else:
            row_vector += value * vectors[col][country_code][0]

    return row_vector

In [31]:
participants = df.apply(lambda x: process_row(x), axis=1).to_dict()
participants = {
    p: {'embedding': v, 'projections':{}, 'predictions':{}, 'gold':{}} for p, v in participants.items()
}

In [32]:
with open('participants.pkl', 'wb') as f:
    pickle.dump(participants, f, protocol=pickle.HIGHEST_PROTOCOL)